### Importing necessary libraries

In [43]:
import os
import getpass
from langchain.agents import Tool
from langchain.tools import BaseTool
from langchain.agents import initialize_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

### Configuring gemini model(from langchain)

In [44]:
os.environ["GOOGLE_API_KEY"] = "your gemini key here"

In [45]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Provide your Google API key here")

In [46]:
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

### Defining the tools (validation prompt and travel cost prediction)

In [47]:
def validate_prompt(prompt):
    query=(
        f"Your task is to validate the {prompt} if user has provided these info or not",
        "Information to validate: Budget, Duration, season, travel style (like adventure, family, cultural), food prefrence, international destination or not, departure location",
        "Just validate it on these info ask nothing else"
    )
    validated=llm.invoke(query)
    return validated

In [48]:
prompt_validation_tool=Tool(
    name='Prompt Validation Tool',
    func=validate_prompt,
    description='The task it to validate if the prompt contains all the necessary information to predict travel cost or not'
)

In [49]:
def predict_travel_cost(prompt):
    query = (
        f"Your task is to provide top 5 location to visit based on {prompt}, nothing else."
    )
    response = llm.invoke(query)  
    return response

In [50]:
cost_prediction_tool=Tool(
    name='Travel Cost Prediction Tool',
    func=predict_travel_cost,
    description='The task is to recommend places based on the prompt provided by the user'
)

In [51]:
def follow_up_question(prediction_result):
    query=(
        f"Your task is to give answers to the follow up questions asked by the user on {prediction_result}"
    )
    response=llm.invoke(query)
    return response

In [52]:
follow_up_question_tool=Tool(
    name='Follow up questions Tool',
    func=follow_up_question,
    description='The task is to provide answers to the questions asked by the user based on the recommendation result'
)

### Creating the agent 

In [53]:
tools_recommendation=[prompt_validation_tool,cost_prediction_tool,follow_up_question_tool]
tools_question=[follow_up_question_tool]
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

In [54]:
conversational_agent_recommendation=initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools_recommendation,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory
)

conversational_agent_question=initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools_question,
    llm=llm,
    verbose=True,
    max_iteration=3,
    memory=memory
)

### Content generation function

In [55]:
def validate_and_predict():
    while True:
        prompt = input()
       
        validation_result = conversational_agent_recommendation.run(f"Validate the prompt if this contains all the info as provided to the tool: {prompt}")
        if "not" in validation_result:
            print(f"Prompt validation failed. Please provide more details. {validation_result}")
        else:
            prediction_result = conversational_agent_recommendation.run(f"Now recommend places according to tool: {prompt} ")
            return prediction_result

# prompt=""
# validation_result = conversational_agent.run(f"Validate the prompt if this contains all the info as provided to the tool: {prompt}")
# prediction_result = conversational_agent.run(f"Now recommend places according to tool: {prompt} ")
# prediction_resultrecommend me places to visit for adventure during summer from rishikesh no international travel with a budget of 50k for a week and the food preference is veg only

In [ ]:
result = validate_and_predict()
print(result)



> Entering new AgentExecutor chain...
```json
{
  "action": "Prompt Validation Tool",
  "action_input": "recommend me places to visit for adventure during summer from rishikesh no international travel with a budget of 50k for a week and the food preference is veg only"
}
```
Observation: content='The information provided is sufficient for validation. We have:\n\n* **Budget:** 50k\n* **Duration:** 1 week\n* **Season:** Summer\n* **Travel style:** Adventure\n* **Food preference:** Veg\n* **International destination:** No\n* **Departure location:** Rishikesh \n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category'